#### EXTRACT

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/Sales Transaction v.4a.csv")
df.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,17490.0,United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,13069.0,United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,13069.0,United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069.0,United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069.0,United Kingdom


In [3]:
df.shape

(536350, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TransactionNo  536350 non-null  object 
 1   Date           536350 non-null  object 
 2   ProductNo      536350 non-null  object 
 3   ProductName    536350 non-null  object 
 4   Price          536350 non-null  float64
 5   Quantity       536350 non-null  int64  
 6   CustomerNo     536295 non-null  float64
 7   Country        536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB


#### Desc of DataFrame

1. TransactionNo (categorical): a six-digit unique number that defines each transaction. The letter “C” in the code indicates a cancellation.
1. Date (numeric): the date when each transaction was generated.
1. ProductNo (categorical): a five or six-digit unique character used to identify a specific product.
1. Product (categorical): product/item name.
1. Price (numeric): the price of each product per unit in pound sterling (£).
1. Quantity (numeric): the quantity of each product per transaction. Negative values related to cancelled transactions.
1. CustomerNo (categorical): a five-digit unique number that defines each customer.
1. Country (categorical): name of the country where the customer resides.

#### TRANSFORM

In [5]:
df = df.rename(columns={"TransactionNo": "transaction_id", "Date": "date", "ProductNo": "product_id",
                        "ProductName": "name", "Price": "price", "Quantity": "quantity", 
                        "CustomerNo": "customer_id", "Country": "country"})

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536350 entries, 0 to 536349
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   transaction_id  536350 non-null  object 
 1   date            536350 non-null  object 
 2   product_id      536350 non-null  object 
 3   name            536350 non-null  object 
 4   price           536350 non-null  float64
 5   quantity        536350 non-null  int64  
 6   customer_id     536295 non-null  float64
 7   country         536350 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 32.7+ MB


In [7]:
df = df.drop_duplicates()
df.shape

(531150, 8)

In [8]:
df.isnull().sum()

transaction_id     0
date               0
product_id         0
name               0
price              0
quantity           0
customer_id       55
country            0
dtype: int64

In [9]:
df.loc[
    df["customer_id"].isnull()
].head()

,transaction_id,date,product_id,name,price,quantity,customer_id,country
6511,C581406,12/8/2019,46000M,Polyester Filler Pad 45x45cm,6.19,-240,NaN,United Kingdom
6512,C581406,12/8/2019,46000S,Polyester Filler Pad 40x40cm,6.19,-300,NaN,United Kingdom
90098,C575153,11/8/2019,22947,Wooden Advent Calendar Red,44.25,-1,NaN,United Kingdom
102671,C574288,11/3/2019,22178,Victorian Glass Hanging T-Light,25.37,-1,NaN,United Kingdom
117263,C573180,10/28/2019,23048,Set Of 10 Lanterns Fairy Light Star,14.50,-1,NaN,United Kingdom


In [11]:
customer_nan = df.loc[
    df["customer_id"].isnull()
].index

In [12]:
df.drop(customer_nan, inplace=True)

In [13]:
df.shape

(531095, 8)

#### Dealing with Unmatched Data

In [14]:
df.loc[
    df["transaction_id"].str.contains("C")
].shape

(8494, 8)

In [15]:
c_trans = df.loc[
    df["transaction_id"].str.contains("C")
].index

In [16]:
df.drop(c_trans, inplace=True)

In [17]:
df.loc[
    df["quantity"] <= 0
].shape

(0, 8)

In [18]:
df.shape

(522601, 8)

#### Data Type Casting

In [19]:
df = df.astype({"transaction_id": int, "customer_id": int})
df["date"] = pd.to_datetime(df["date"], format="%m/%d/%Y")

In [20]:
df.dtypes

transaction_id             int64
date              datetime64[ns]
product_id                object
name                      object
price                    float64
quantity                   int64
customer_id                int64
country                   object
dtype: object

In [21]:
product_unique = df.loc[
    ~df[["product_id", "name", "price"]].duplicated()
]

In [22]:
product_unique.sort_values(by=["name", "date"]).head()

,transaction_id,date,product_id,name,price,quantity,customer_id,country
507047,538877,2018-12-14,22418,10 Colour Spaceboy Pen,12.82,2,14877,United Kingdom
491632,540154,2019-01-05,22418,10 Colour Spaceboy Pen,11.95,2,12154,United Kingdom
130922,572035,2019-10-20,22418,10 Colour Spaceboy Pen,10.99,144,14646,Netherlands
94877,574856,2019-11-07,22418,10 Colour Spaceboy Pen,12.77,2,17856,United Kingdom
52165,578067,2019-11-22,22418,10 Colour Spaceboy Pen,11.92,1,12067,United Kingdom


In [23]:
df[["date","product_id", "name", "price"]].sort_values(by=["name", "date"]).head()

,date,product_id,name,price
533683,2018-12-01,22418,10 Colour Spaceboy Pen,11.12
534531,2018-12-01,22418,10 Colour Spaceboy Pen,11.12
534759,2018-12-01,22418,10 Colour Spaceboy Pen,11.95
535239,2018-12-01,22418,10 Colour Spaceboy Pen,11.12
531338,2018-12-02,22418,10 Colour Spaceboy Pen,11.12


#### Sum the quantity if identical transactions occur

In [24]:
type(df.groupby(["transaction_id", "date", "product_id", "name", "price", "customer_id", "country"])["quantity"])

pandas.core.groupby.generic.SeriesGroupBy

In [25]:
df.loc[
    (df["transaction_id"] == 579171) &
    (df["product_id"] == "22897")
]

,transaction_id,date,product_id,name,price,quantity,customer_id,country
35015,579171,2019-11-28,22897,Oven Mitt Apples Design,6.19,4,13030,United Kingdom
35040,579171,2019-11-28,22897,Oven Mitt Apples Design,6.19,2,13030,United Kingdom


In [26]:
df = df.groupby(["transaction_id", "date", "product_id", "name", "price", "customer_id", "country"])["quantity"].sum().reset_index()

In [27]:
df.loc[
    (df["transaction_id"] == 579171) &
    (df["product_id"] == "22897")
]

,transaction_id,date,product_id,name,price,customer_id,country,quantity
484529,579171,2019-11-28,22897,Oven Mitt Apples Design,6.19,13030,United Kingdom,6


#### Enriching Data

In [28]:
df["year"] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week
df["day"] = df['date'].dt.day
df["day_name"] = df['date'].dt.day_name()

In [29]:
country_id = df["country"].str.upper().str.slice(stop=3) + df["country"].str.len().astype(str)
df["country_id"] = country_id

In [30]:
df.dtypes

transaction_id             int64
date              datetime64[ns]
product_id                object
name                      object
price                    float64
customer_id                int64
country                   object
quantity                   int64
year                       int64
quarter                    int64
month                      int64
week                      UInt32
day                        int64
day_name                  object
country_id                object
dtype: object

#### Load

In [32]:
from sqlalchemy import create_engine, text
import psycopg2

In [33]:
try:
    conn = psycopg2.connect("host = localhost dbname = postgres user = sakar_ password = sakar10") #connecting to the default database
except psycopg2.Error as e :
    print("error, couldnt connect to database")
    print(e)
conn.set_session(autocommit = True) # any changes we make from here will be updated automatically in the database
curr = conn.cursor()

In [34]:
curr.execute("CREATE DATABASE sales_db")   # creating our database in postgres
conn.close()   # closing connection to the default database

In [35]:
try:
    conn = psycopg2.connect("host = localhost dbname = sales_db user = sakar_ password = sakar10") # connecting to our new database
except psycopg2.Error as e :
    print("error, couldnt connect to database")
    print(e)
conn.set_session(autocommit = True)
curr = conn.cursor() # creating a cursor to be able to communicate to our database

#### Generate Keys for all dimensional tables

In [36]:
df_customer = df[["customer_id"]].drop_duplicates().sort_values(by="customer_id")
df_customer = df_customer.reset_index(drop=True)
df_customer["customer_key"] = df_customer.index
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718 entries, 0 to 4717
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   customer_id   4718 non-null   int64
 1   customer_key  4718 non-null   int64
dtypes: int64(2)
memory usage: 73.8 KB


In [37]:
df_transaction = df[["transaction_id"]].drop_duplicates().sort_values(by="transaction_id")
df_transaction = df_transaction.reset_index(drop=True)
df_transaction["transaction_key"] = df_transaction.index
df_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19789 entries, 0 to 19788
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   transaction_id   19789 non-null  int64
 1   transaction_key  19789 non-null  int64
dtypes: int64(2)
memory usage: 309.3 KB


In [38]:
df_date = df[["date", "year", "quarter", "month", "week", "day", "day_name"]].drop_duplicates().sort_values(by="date")
df_date = df_date.reset_index(drop=True)
df_date["date_key"] = df_date.index
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 305 entries, 0 to 304
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      305 non-null    datetime64[ns]
 1   year      305 non-null    int64         
 2   quarter   305 non-null    int64         
 3   month     305 non-null    int64         
 4   week      305 non-null    UInt32        
 5   day       305 non-null    int64         
 6   day_name  305 non-null    object        
 7   date_key  305 non-null    int64         
dtypes: UInt32(1), datetime64[ns](1), int64(5), object(1)
memory usage: 18.3+ KB


In [39]:
df_country = df[["country", "country_id"] ].drop_duplicates().sort_values(by="country_id")
df_country = df_country.reset_index(drop=True)
df_country["country_key"] = df_country.index
df_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country      38 non-null     object
 1   country_id   38 non-null     object
 2   country_key  38 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ KB


In [40]:
df_product = df[['product_id', "name", "price"]]
unique_product = df_product.drop_duplicates().sort_values(by=["name", "price"])
unique_product = unique_product.reset_index(drop=True)
unique_product["product_key"] = unique_product.index

df_unique_product = pd.merge(df_product, unique_product, on=["product_id", "name", "price"]).sort_values(by=["name", "price"]).drop_duplicates()
df_unique_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23498 entries, 492120 to 401712
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   product_id   23498 non-null  object 
 1   name         23498 non-null  object 
 2   price        23498 non-null  float64
 3   product_key  23498 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 917.9+ KB


#### Join to all fact_table

In [41]:
result = pd.merge(df, df_customer, on="customer_id")
result = pd.merge(result, df_transaction, on="transaction_id")
result = pd.merge(result, df_date, on=["date", "year", "quarter", "month", "week", "day", "day_name"])
result = pd.merge(result,df_country, on=["country_id", "country"])
result = pd.merge(result, df_unique_product, on=["product_id", "name", "price"])

In [42]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517897 entries, 0 to 517896
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   transaction_id   517897 non-null  int64         
 1   date             517897 non-null  datetime64[ns]
 2   product_id       517897 non-null  object        
 3   name             517897 non-null  object        
 4   price            517897 non-null  float64       
 5   customer_id      517897 non-null  int64         
 6   country          517897 non-null  object        
 7   quantity         517897 non-null  int64         
 8   year             517897 non-null  int64         
 9   quarter          517897 non-null  int64         
 10  month            517897 non-null  int64         
 11  week             517897 non-null  UInt32        
 12  day              517897 non-null  int64         
 13  day_name         517897 non-null  object        
 14  country_id       517

#### Add col and Load dim table to postgres

In [43]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4718 entries, 0 to 4717
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   customer_id   4718 non-null   int64
 1   customer_key  4718 non-null   int64
dtypes: int64(2)
memory usage: 73.8 KB
